In [1]:
# importamos las librerías que necesitamos

# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Visualización
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Evaluar linealidad de las relaciones entre las variables
# y la distribución de las variables
# ------------------------------------------------------------------------------
from scipy.stats import shapiro, kstest
import scipy.stats as stats
from scipy.stats import shapiro, poisson, chisquare, expon, kstest

# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

# Gestión de los warnings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv("Datos/world_data_full_apply_nulos.csv", index_col=0)
df.head(2)

,country,density,abbreviation,agricultural_land,land_area,armed_forces_size,birth_rate,calling_code,capital/major_city,co2-emissions,cpi,cpi_change,currency-code,fertility_rate,forested_area,gasoline_price,gdp,gross_primary_education_enrollment,gross_tertiary_education_enrollment,infant_mortality,largest_city,life_expectancy,maternal_mortality_ratio,minimum_wage,official_language,out_of_pocket_health_expenditure,physicians_per_thousand,population,population_labor_force_participation,tax_revenue,total_tax_rate,unemployment_rate,urban_population,latitude,longitude,continent
0,Afghanistan,60.0,AF,58.1,652.230,323.0,32.49,93.0,Kabul,8.672,149.90,2.3,AFN,4.47,2.1,0.70,1.910135e+10,104.0,9.7,47.9,Kabul,64.5,638.0,0.43,Pashto,78.4,0.28,NaN,48.9,9.3,71.4,11.12,NaN,33.939110,67.709953,Asia
1,Albania,105.0,AL,43.1,28.748,9.0,11.78,355.0,Tirana,4.536,119.05,1.4,ALL,1.62,28.1,1.36,1.527808e+10,107.0,55.0,7.8,Tirana,78.5,15.0,1.12,Albanian,56.9,1.20,NaN,55.7,18.6,36.6,12.33,NaN,41.153332,20.168331,Europe


1.- Ejercicios intervalo de confianza. Al igual que en el caso del ejercicio de la clase, piensa un motivo por el que sería interesante calcular estos intervalos de confianza para la extracción de insights.

    a.- Intervalo de Confianza para la Esperanza de Vida Contexto: Trabajas en una organización de salud global y deseas estimar el intervalo de confianza al 95% para la esperanza de vida promedio de cada continente. Nota: Cuidado porque puede que os salgan algunos nulos, genera un código para que no salgan nulos en los intervalos de confianza.

    b.- Intervalo de Confianza para el PIB Contexto: Eres una economista y quieres calcular el intervalo de confianza al 90% para el Producto Interno Bruto (PIB) de los continentes.

2.- Prueba de Hipótesis

    a.- Test de Normalidad :Verifica si la variable "life_expectancy" (esperanza de vida) sigue una distribución normal usando el test de Shapiro-Wilk o Kolmogorov-Smirnov según el tamaño de la muestra.

    b.- Test de Igualdad de Varianzas: Comprueba si la varianza del PIB (GDP) es igual en países de Europa y África usando el test de Bartlett (si los datos son normales) o Levene (si no son normales).

    c.- Comparación de Medias (t-Test de Student o Mann-Whitney U): ¿La esperanza de vida difiere significativamente entre países de Europa y Asia?

    d.- Prueba de Hipótesis sobre Proporciones: ¿Hay una diferencia significativa en la tasa de desempleo entre los países de Europa y América?

    e.- Comparación de Medias en Precios de Gasolina : ¿El precio de la gasolina (gasoline_price) es significativamente diferente entre países con un PIB alto y PIB bajo?

1.- Ejercicios intervalo de confianza. Al igual que en el caso del ejercicio de la clase, piensa un motivo por el que sería interesante calcular estos intervalos de confianza para la extracción de insights.

    a.- Intervalo de Confianza para la Esperanza de Vida Contexto: Trabajas en una organización de salud global y deseas estimar el intervalo de confianza al 95% para la esperanza de vida promedio de cada continente. Nota: Cuidado porque puede que os salgan algunos nulos, genera un código para que no salgan nulos en los intervalos de confianza.

In [3]:
df.columns

Index(['country', 'density', 'abbreviation', 'agricultural_land', 'land_area',
       'armed_forces_size', 'birth_rate', 'calling_code', 'capital/major_city',
       'co2-emissions', 'cpi', 'cpi_change', 'currency-code', 'fertility_rate',
       'forested_area', 'gasoline_price', 'gdp',
       'gross_primary_education_enrollment',
       'gross_tertiary_education_enrollment', 'infant_mortality',
       'largest_city', 'life_expectancy', 'maternal_mortality_ratio',
       'minimum_wage', 'official_language', 'out_of_pocket_health_expenditure',
       'physicians_per_thousand', 'population',
       'population_labor_force_participation', 'tax_revenue', 'total_tax_rate',
       'unemployment_rate', 'urban_population', 'latitude', 'longitude',
       'continent'],
      dtype='object')

In [4]:
df

,country,density,abbreviation,agricultural_land,land_area,armed_forces_size,birth_rate,calling_code,capital/major_city,co2-emissions,cpi,cpi_change,currency-code,fertility_rate,forested_area,gasoline_price,gdp,gross_primary_education_enrollment,gross_tertiary_education_enrollment,infant_mortality,largest_city,life_expectancy,maternal_mortality_ratio,minimum_wage,official_language,out_of_pocket_health_expenditure,physicians_per_thousand,population,population_labor_force_participation,tax_revenue,total_tax_rate,unemployment_rate,urban_population,latitude,longitude,continent
0,Afghanistan,60.0,AF,58.1,652.230,323.0,32.49,93.0,Kabul,8.672,149.90,2.3,AFN,4.47,2.1,0.70,1.910135e+10,104.0,9.7,47.9,Kabul,64.5,638.0,0.43,Pashto,78.4,0.28,NaN,48.9,9.3,71.4,11.12,NaN,33.939110,67.709953,Asia
1,Albania,105.0,AL,43.1,28.748,9.0,11.78,355.0,Tirana,4.536,119.05,1.4,ALL,1.62,28.1,1.36,1.527808e+10,107.0,55.0,7.8,Tirana,78.5,15.0,1.12,Albanian,56.9,1.20,NaN,55.7,18.6,36.6,12.33,NaN,41.153332,20.168331,Europe
2,Algeria,18.0,DZ,17.4,NaN,317.0,24.28,213.0,Algiers,150.006,151.36,2.0,DZD,3.02,0.8,0.28,1.699882e+11,109.9,51.4,20.1,Algiers,76.7,112.0,0.95,Arabic,28.1,1.72,NaN,41.2,37.2,66.1,11.70,NaN,28.033886,1.659626,Africa
3,Andorra,164.0,AD,40.0,468.000,NaN,7.20,376.0,Andorra la Vella,469.000,NaN,NaN,EUR,1.27,34.0,1.51,3.154058e+09,106.4,NaN,2.7,Andorra la Vella,NaN,NaN,6.63,Catalan,36.4,3.33,77.142,NaN,NaN,NaN,NaN,67.873,42.506285,1.521801,Europe
4,Angola,26.0,AO,47.5,NaN,117.0,40.73,244.0,Luanda,34.693,261.73,17.1,AOA,5.52,46.3,0.97,9.463542e+10,113.5,9.3,51.6,Luanda,60.8,241.0,0.71,Portuguese,33.4,0.21,NaN,77.5,9.2,49.1,6.89,NaN,-11.202692,17.873887,Africa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,Venezuela,32.0,VE,24.5,912.050,343.0,17.88,58.0,Caracas,164.175,NaN,254.9,VED,2.27,52.7,0.00,4.823593e+11,97.2,79.3,21.4,Caracas,72.1,125.0,0.01,Spanish,45.8,1.92,NaN,59.7,NaN,73.3,8.80,NaN,6.423750,-66.589730,South America
191,Vietnam,314.0,VN,39.3,331.210,522.0,16.75,84.0,Hanoi,192.668,163.52,2.8,VND,2.05,48.1,0.80,2.619212e+11,110.6,28.5,16.5,Ho Chi Minh City,75.3,43.0,0.73,Vietnamese,43.5,0.82,NaN,77.4,19.1,37.6,2.01,NaN,14.058324,108.277199,Asia
192,Yemen,56.0,YE,44.6,527.968,40.0,30.45,967.0,Sanaa,10.609,157.58,8.1,YER,3.79,1.0,0.92,2.691440e+10,93.6,10.2,42.9,Sanaa,66.1,164.0,NaN,Arabic,81.0,0.31,NaN,38.0,NaN,26.6,12.91,NaN,15.552727,48.516388,Asia
193,Zambia,25.0,ZM,32.1,752.618,16.0,36.19,260.0,Lusaka,5.141,212.31,9.2,ZMW,4.63,65.2,1.40,2.306472e+10,98.7,4.1,40.4,Lusaka,63.5,213.0,0.24,English,27.5,1.19,NaN,74.6,16.2,15.6,11.43,NaN,-13.133897,27.849332,Africa


In [5]:
media = df["life_expectancy"].mean()
error = stats.sem(df["life_expectancy"], nan_policy="omit")
grados_libertad = len(df["life_expectancy"]) -1
nivel_confianza = 0.95
valor_critico = stats.t.ppf((1 + nivel_confianza)/2, df = grados_libertad)
salto = valor_critico * error
limite_superior = media + salto
limite_inferior = media - salto
print(f"El intervalo de confianza del 95% para la esperanza de vida es entre {np.round(limite_inferior, 2)} y {np.round(limite_superior, 2)}")

El intervalo de confianza del 95% para la esperanza de vida es entre 71.2 y 73.36


In [ ]:

for continent in df["continent"].unique():
    df_p = df[df["continent"] == continent]
    media = df_p["life_expectancy"].mean()
    error = stats.sem(df_p["life_expectancy"], nan_policy="omit")
    grados_libertad = len(df_p["life_expectancy"])-1
    nivel_confianza = 0.95
    valor_critico = stats.t.ppf((1 + nivel_confianza) / 2, df=grados_libertad)
    salto = valor_critico * error
    limite_sup = media + salto
    limite_inf = media - salto
    print(continent)
    print(limite_inf, "-", limite_sup)
    print("---------------------------------------------------------")

Asia
72.32510366298024 - 75.23853270065612
---------------------------------------------------------
Europe
78.0907284332469 - 80.31392272954382
---------------------------------------------------------
Africa
62.05513364839719 - 65.3948663516028
---------------------------------------------------------
Central America
73.04076946098671 - 76.33923053901326
---------------------------------------------------------
South America
72.98698407830678 - 76.64378515246243
---------------------------------------------------------
Oceania
67.46625079717728 - 74.78374920282269
---------------------------------------------------------
North America
58.59945194806541 - 101.8005480519346
---------------------------------------------------------
nan
nan - nan
---------------------------------------------------------


    b.- Intervalo de Confianza para el PIB Contexto: Eres una economista y quieres calcular el intervalo de confianza al 90% para el Producto Interno Bruto (PIB) de los continentes.

In [7]:

for continent in df["continent"].unique():
    df_p = df[df["continent"] == continent]
    media = df_p["gdp"].mean()
    error = stats.sem(df_p["gdp"], nan_policy="omit")
    grados_libertad = len(df_p["gdp"])-1
    nivel_confianza = 0.95
    valor_critico = stats.t.ppf((1 + nivel_confianza) / 2, df=grados_libertad)
    salto = valor_critico * error
    limite_sup = media + salto
    limite_inf = media - salto
    print(continent)
    print(limite_inf, "-", limite_sup)
    print("---------------------------------------------------------")

Asia
-96257119210.44543 - 1770647650512.1729
---------------------------------------------------------
Europe
236781303959.969 - 736715076648.5093
---------------------------------------------------------
Africa
21895327332.592236 - 70265790955.14362
---------------------------------------------------------
Central America
-41293592517.24237 - 218253447923.44238
---------------------------------------------------------
South America
536550544.17700195 - 596330820553.3615
---------------------------------------------------------
Oceania
-97585015425.53658 - 331162126930.10803
---------------------------------------------------------
North America
-113518619021528.45 - 136682744651048.45
---------------------------------------------------------
nan
nan - nan
---------------------------------------------------------
